In [1]:
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pandas as pd
import seaborn as sns
from tslearn.clustering import TimeSeriesKMeans
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow import keras
import seaborn as sns
import pickle
import functools
import fastdtw
import scipy.spatial.distance as sd
from datetime import datetime
import jsonpickle
import json
import pickle

2022-02-06 20:50:01.570785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-06 20:50:01.570813: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Import datasets

In [2]:
consumption_df = pd.read_csv('Final Datasets/energy_consumption.csv')
clustered_df = pd.read_csv('Tables/clustered.csv')
generation_df = pd.read_csv('Final Datasets/energy_generation.csv')

In [3]:
scenarios_data = [[1, 4, 6, 'Yes'], [2, 1, 6, 'Yes'], [3, 4, 1, 'No'], [4, 1, 1, 'No']]
scenarios_df = pd.DataFrame(scenarios_data, columns = ['Scenarios', 'No. of subsets', 'No. of features', 'Smart devices included in modeling?']).set_index('Scenarios')

#### Converting Dataset to House-wise dictionary

In [4]:
house_wise_consumption = dict()
for house_number in consumption_df['House Number'].unique():
    house_data = consumption_df[consumption_df['House Number'] == house_number]
    house_wise_consumption[house_number] = house_data.copy()
#house_wise_consumption[1]    

In [5]:
class House:
    cols = ['energy_kWh', 'electronics workbench energy', 'network equipment energy',
            'home office energy', 'RAE home office energy', 'RAE Mobile Phone Charger Energy']
    def __init__(self, house):
        self.house = house
        self.consumption_df = house_wise_consumption[house].copy()[House.cols].astype(float)
        self.train_dates_df = house_wise_consumption[house]['timestamp']
        self.prepare_dataset(n_future, n_past)
    
    def __repr__(self):
        return str(self.house)
    
    def prepare_dataset(self, n_future = 1, n_past = 14):
        #Empty lists to be populated using formatted training data
        trainX = []
        trainY = []
        consumption_np = np.array(self.consumption_df)
        #Reformat input data into a shape: (n_samples x timesteps x n_features)
        #In my example, my self.consumption_df has a shape (12823, 5)
        #12823 refers to the number of data points and 5 refers to the columns (multi-variables).
        for i in range(n_past, len(consumption_np) - n_future +1):
            trainX.append(consumption_np[i - n_past:i, 0:self.consumption_df.shape[1]])
            trainY.append(consumption_np[i + n_future - 1:i + n_future, 0])

        trainX, trainY = np.array(trainX), np.array(trainY)

        #print('trainX shape == {}.'.format(trainX.shape))
        #print('trainY shape == {}.'.format(trainY.shape))
        self.trainX = trainX
        self.trainY = trainY
        
        
class Cluster: 
    def __init__(self, houses):
        self.houses = [House(house) for house in houses]
        self.trainX_list = (list(map(lambda x: x.trainX, self.houses)))
        self.trainY_list = (list(map(lambda x: x.trainY, self.houses)))
        self.train_dates_list = (list(map(lambda x: x.train_dates_df, self.houses)))
        self.consumption_df_list = (list(map(lambda x: x.consumption_df, self.houses)))
        self.prepare_dataset()
        
    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)
    
    def prepare_dataset(self):
        
        self.train_dates = np.concatenate(self.train_dates_list, axis=0)
        ## TODO - Replace concatenate with for loop or sort by time
        self.X = np.concatenate(self.trainX_list, axis=0)
        self.Y = np.concatenate(self.trainY_list, axis=0)
    
    def prepare_model(self):
        model = Sequential()
        model.add(LSTM(128, activation='relu', input_shape=(self.X.shape[1], self.X.shape[2]), return_sequences=True))
        model.add(LSTM(32, activation='relu', return_sequences=True))
        model.add(LSTM(16, activation='relu', return_sequences=False))
        model.add(Dense(self.Y.shape[1]))
        model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])
        model.summary()
        return model
    
    def train_model(self):
        model = self.prepare_model()
        history = model.fit(self.X, self.Y, epochs=1, batch_size=16, validation_split=0.3, verbose=1)
        self.history = history
        plt.plot(history.history['loss'], label='Training loss')
        plt.plot(history.history['val_loss'], label='Testing loss')
        self.model = model
        plt.legend()
        return model
    
    def predicted(self):
        model = self.model
        predictions = list(map(lambda x: pd.Series(model.predict(x).reshape(1,-1)[0]), self.trainX_list))
        #return pd.Series(model.predict(self.X).reshape(1,-1)[0])
        
        return dict(zip(self.houses, predictions))
    
    def results(self):
        predicted = self.predicted()
        house_prediction_list_df  = {}
        for house in predicted:
            house_prediction_df = house.consumption_df.iloc[n_past:].copy().reset_index(drop=True)
            house_prediction_df['timestamp'] = house.train_dates_df.iloc[n_past:].copy().reset_index(drop=True)
            house_prediction_df['energy_predicted'] = predicted[house]
            ### Anomaly Detection
            energy_stddev = house_prediction_df['energy_kWh'].std()
            house_prediction_df['Energy Consumption Error'] = house_prediction_df['energy_predicted'] - house_prediction_df['energy_kWh']
            house_prediction_df['Anomaly detected'] = house_prediction_df['Energy Consumption Error'].apply(lambda x: False if abs(x) < 1.5*energy_stddev else abs(x))
            ### Energy Deficit Calculation
            house_prediction_df['timestamp'] = pd.DatetimeIndex(house_prediction_df['timestamp'])
            generation_df['timestamp'] = pd.DatetimeIndex(generation_df['timestamp'])
            house_prediction_df = house_prediction_df.set_index('timestamp').join(generation_df.set_index('timestamp'))
            house_prediction_df['timestamp'] = house_prediction_df.index
            house_prediction_df.reset_index(drop=True, inplace=True)
            house_prediction_df['Total Generated Energy'] = house_prediction_df['Wind Energy']+ house_prediction_df['Solar Energy']
            house_prediction_df['Predicted Energy Deficit'] = house_prediction_df['Total Generated Energy'] - house_prediction_df['energy_predicted']
            house_prediction_df['Energy Deficit Flag'] = house_prediction_df['Predicted Energy Deficit'].apply(lambda x: False if x>0 else abs(x))
            house_prediction_list_df[house.house] = house_prediction_df
        return house_prediction_list_df

#### Time-Series LSTM modeling

In [6]:
n_past = 14
n_future = 1

In [7]:
### Load Cluster Object

In [8]:
with open('Models/Objects/zero_cluster', 'rb') as f:
    zero_cluster = pickle.load(f)

2022-02-06 20:50:03.903327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-06 20:50:03.903352: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-06 20:50:03.903376: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (rishabh-Inspiron): /proc/driver/nvidia/version does not exist
2022-02-06 20:50:03.903546: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# with open("Models/Objects/zero_cluster.json", "r") as f:
#     zero_cluster = json.load(f)
#     zero_cluster = jsonpickle.decode(zero_cluster)

In [10]:
results = zero_cluster.results()

In [11]:
results.keys()

dict_keys([1, 4, 13, 14, 18, 19, 28])

In [12]:
results[1]

,energy_kWh,electronics workbench energy,network equipment energy,home office energy,RAE home office energy,RAE Mobile Phone Charger Energy,energy_predicted,Energy Consumption Error,Anomaly detected,Wind Energy,Solar Energy,timestamp,Total Generated Energy,Predicted Energy Deficit,Energy Deficit Flag
0,0.503,0.007318,0.019336,0.014161,0.068592,0.000235,0.790017,0.287017,False,0.320828,0.6870,2012-06-01 15:00:00,1.007828,0.217811,False
1,0.556,0.007461,0.019334,0.014586,0.075476,0.000255,0.673105,0.117105,False,0.321386,0.3546,2012-06-01 16:00:00,0.675986,0.002882,False
2,0.925,0.008455,0.019319,0.017543,0.123401,0.000398,0.768880,-0.156120,False,0.254128,0.1350,2012-06-01 17:00:00,0.389128,-0.379752,0.379752
3,0.713,0.007884,0.019328,0.015844,0.095867,0.000316,1.346284,0.633284,False,0.186636,0.1320,2012-06-01 18:00:00,0.318636,-1.027648,1.027648
4,0.597,0.007571,0.019332,0.014915,0.080801,0.000271,1.264435,0.667435,False,0.187290,0.0492,2012-06-01 19:00:00,0.236490,-1.027945,1.027945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29212,2.101,0.011621,0.019273,0.026968,0.276137,0.000851,2.692356,0.591356,False,0.525448,0.0000,2015-10-02 20:00:00,0.525448,-2.166908,2.166908
29213,1.646,0.010396,0.019291,0.023322,0.217043,0.000675,1.892630,0.246630,False,0.525448,0.0000,2015-10-02 21:00:00,0.525448,-1.367182,1.367182
29214,1.319,0.009516,0.019304,0.020701,0.174573,0.000549,1.547507,0.228507,False,0.525448,0.0000,2015-10-02 22:00:00,0.525448,-1.022059,1.022059
29215,1.473,0.009930,0.019298,0.021935,0.194574,0.000609,1.323973,-0.149027,False,0.526383,0.0000,2015-10-02 23:00:00,0.526383,-0.797591,0.797591
